In [1]:
# import libraries
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pathlib
from pathlib import Path
import os
import seaborn as sns
import datetime
import shapely
from shapely.geometry import shape, Point, LineString, Polygon

# set the working directory
BASE_DIR = Path.cwd()
# print(BASE_DIR)

#set the project directory
project_folder = BASE_DIR.parent.parent
print(project_folder)

/tmp/pycharm_project_316/BEAM_Freight


/home/goyal/.virtualenvs/venv_lbl_gce/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_88299/4263173763.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/lates

In [2]:
# Upload output in the respective bucket storage in the following path
# "gs://beam-core-outputs/output/city_name/simulation_name/Output/*" or
# "gs://beam-core-outputs/output/city_name/simulation_name/Output/plot/*" or

from google.cloud import storage
# Upload file to Google Cloud service
def upload_blob(_bucket_name, _source_file_name, _destination_blob_name):
    """Uploads a file to the bucket."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"
    # The path to your file to upload
    # source_file_name = "local/path/to/file"
    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"
    storage_client = storage.Client()
    bucket = storage_client.bucket(_bucket_name)
    blob = bucket.blob(_destination_blob_name)
    # Optional: set a generation-match precondition to avoid potential race conditions
    # and data corruptions. The request to upload is aborted if the object's
    # generation number does not match your precondition. For a destination
    # object that does not yet exist, set the if_generation_match precondition to 0.
    # If the destination object already exists in your bucket, set instead a
    # generation-match precondition using its generation number.
    generation_match_precondition = 0
    blob.upload_from_filename(_source_file_name, if_generation_match=generation_match_precondition)
    print(
        f"File {_source_file_name} uploaded to {_destination_blob_name}."
    )

def delete_blob(_bucket_name, _blob_name):
    """Deletes a blob from the bucket."""
    # bucket_name = "your-bucket-name"
    # blob_name = "your-object-name"

    storage_client = storage.Client()

    bucket = storage_client.bucket(_bucket_name)
    blob = bucket.blob(_blob_name)
    generation_match_precondition = None

    # Optional: set a generation-match precondition to avoid potential race conditions
    # and data corruptions. The request to delete is aborted if the object's
    # generation number does not match your precondition.
    blob.reload()  # Fetch blob metadata to use in generation_match_precondition.
    generation_match_precondition = blob.generation

    blob.delete(if_generation_match=generation_match_precondition)

    print(f"Blob {_blob_name} deleted.")

In [3]:
# Basic Reference Points
city_name="austin"
gcloud_bucket = "gs://beam-core-outputs/output/austin"
iter_no = "0"
linkstats_file_path = f"{iter_no}.linkstats.csv.gz"
network_file_path = "network.csv.gz"
event_files_path = f"{iter_no}.events.csv.gz"
simulation_name = "austin-base-with-freight-2018__2023-04-14_16-11-13_yoq"
analysis_type = "Freight"
#Check if the folder exist, if not create it.
try:
    pathlib.Path(project_folder.joinpath("Output", city_name, simulation_name, analysis_type, "plot")).mkdir(parents=True, exist_ok=False)
except:
    pass

In [ ]:
# read the network file and convert into featureclass
# df_network = pd.read_csv(project_folder.joinpath("Data", city_name, simulation_name,"network.csv.gz"), compression="gzip", low_memory=True)
df_network = pd.read_csv(f"{gcloud_bucket}/{simulation_name}/{network_file_path}", compression="gzip", low_memory=True)
df_network['geometry'] = df_network.apply(lambda x: LineString([(x['fromLocationX'], x['fromLocationY']) , (x['toLocationX'], x['toLocationY'])]), axis = 1)
gdf_network = gpd.GeoDataFrame(df_network, geometry=df_network["geometry"])
gdf_network = gdf_network.set_crs(26910,allow_override=True)
gdf_network.to_crs(4326, inplace=True) # convert to WGS1984
# keep only selected columns
col_to_keep = ["linkId",  "linkLength", "linkFreeSpeed", "linkCapacity", "numberOfLanes", "linkModes", "attributeOrigId", "attributeOrigType", "geometry"]

fig, ax=plt.subplots(figsize=(30,30))
gdf_network.to_crs(4326, inplace=True)
gdf_network.plot(ax=ax,edgecolor='black',alpha= 0.4,)
ax.axis('off')
ax.set_title(f'{city_name} Road Network',fontsize=32)
plt.tight_layout()
plt_file_name=f"{city_name}_Road_Network.png"
plt.savefig(project_folder.joinpath("Output", city_name, simulation_name, "Freight", "plot", plt_file_name), dpi=600)

bucket_name = "beam-core-outputs"
source_file_name = project_folder.joinpath("Output", city_name, simulation_name, analysis_type, "plot",plt_file_name)
destination_blob_name = f"output/{city_name}/{simulation_name}/Output/plot/{plt_file_name}"
# Check if the file exist in the bucket. If "Yes", delete
try:
    delete_blob(_bucket_name=bucket_name, _blob_name=destination_blob_name)
except:
    pass
# and upload the file
upload_blob(bucket_name, source_file_name, destination_blob_name)

File /tmp/pycharm_project_316/BEAM_Freight/Output/austin/austin-base-with-freight-2018__2023-04-14_16-11-13_yoq/Freight/plot/austin_Road_Network.png uploaded to output/austin/austin-base-with-freight-2018__2023-04-14_16-11-13_yoq/Output/plot/austin_Road_Network.png.


In [ ]:
# gdf_network = gdf_network[col_to_keep]
gpd_fc_file_name = "beam_network_out.geojson"
gdf_network.to_file(project_folder.joinpath("Output",city_name, simulation_name, analysis_type, gpd_fc_file_name), driver="GeoJSON")

bucket_name = "beam-core-outputs"
source_file_name = project_folder.joinpath("Output", city_name, simulation_name, analysis_type, gpd_fc_file_name)
destination_blob_name = f"output/{city_name}/{simulation_name}/Output/{gpd_fc_file_name}"

# Check if the file exist in the bucket. If "Yes", delete
try:
    delete_blob(_bucket_name=bucket_name, _blob_name=destination_blob_name)
except:
    pass
# and upload the file
upload_blob(bucket_name, source_file_name, destination_blob_name)

In [ ]:
#### clean up beam network and assign county ####

In [ ]:
# intersect the road network with County file
def intersect_county_rdntwrk(_gpdCT, _gpdrdntwrk):
    _gpdCT.to_crs(3857, inplace=True) # set its projection to EPSG:3857
    _gpdrdntwrk.to_crs(3857, inplace=True)
    res_intersection = _gpdrdntwrk.overlay(_gpdCT, how='intersection')
    res_intersection = res_intersection.to_crs(4326)
    return res_intersection

In [ ]:
beam_network_out = gpd.read_file(project_folder.joinpath("Output",city_name, simulation_name, analysis_type, "beam_network_out.geojson"))
austin_boundary = gpd.read_file(project_folder.joinpath("Reference_Data", city_name, "Austin_Counties.geojson"))
# austin_boundary.set_crs(4326, inplace=True)

col_to_keep = ["linkId",  "linkLength", "linkFreeSpeed", "linkCapacity", "numberOfLanes", "linkModes", "attributeOrigId", "attributeOrigType", "geometry","fromNodeId","toNodeId", "name", "txdot_abbr", "fips_code"]
beam_network_splits = (intersect_county_rdntwrk(austin_boundary,beam_network_out)[col_to_keep])
beam_network_splits = beam_network_splits.loc[beam_network_splits["linkLength"]>0.001].copy()

# overlay road network on counties and save the image
fig, ax=plt.subplots(figsize=(30,30))
# austin_boundary.plot(ax=ax,lw=0.7,edgecolor="black")
beam_network_splits.to_crs(4326, inplace=True)
beam_network_splits.plot(ax=ax,column='name', categorical=True, cmap='Pastel2',legend=True,alpha= 0.8,)
ax.axis('off')
ax.set_title(f'BEAM Network Splits',fontsize=32)
plt.tight_layout()
plt_file_name="beam_network_splits.png"
plt.savefig(project_folder.joinpath("Output", city_name, simulation_name, "Freight", "plot", plt_file_name), dpi=600)

bucket_name = "beam-core-outputs"
source_file_name = project_folder.joinpath("Output", city_name, simulation_name, analysis_type, "plot",plt_file_name)
destination_blob_name = f"output/{city_name}/{simulation_name}/Output/plot/{plt_file_name}"
# Check if the file exist in the bucket. If "Yes", delete
try:
    delete_blob(_bucket_name=bucket_name, _blob_name=destination_blob_name)
except:
    pass
# and upload the file
upload_blob(bucket_name, source_file_name, destination_blob_name)

In [ ]:
# export the dataframe
gpd_fc_file_name = "beam_network_by_county.geojson"
beam_network_splits.to_file(project_folder.joinpath("Output",city_name, simulation_name, analysis_type, gpd_fc_file_name), driver="GeoJSON")

bucket_name = "beam-core-outputs"
source_file_name = project_folder.joinpath("Output", city_name, simulation_name, analysis_type, gpd_fc_file_name)
destination_blob_name = f"output/{city_name}/{simulation_name}/Output/{gpd_fc_file_name}"
# Check if the file exist in the bucket. If "Yes", delete
try:
    delete_blob(_bucket_name=bucket_name, _blob_name=destination_blob_name)
except:
    pass
# and upload the file
upload_blob(bucket_name, source_file_name, destination_blob_name)

In [ ]:
df_file_name = "beam_network_by_county.csv"
beam_network_splits.loc[:, ~beam_network_splits.columns.isin(["geometry"])].to_csv(project_folder.joinpath("Output",city_name, simulation_name, analysis_type, df_file_name))

bucket_name = "beam-core-outputs"
source_file_name = project_folder.joinpath("Output", city_name, simulation_name, analysis_type, df_file_name)
destination_blob_name = f"output/{city_name}/{simulation_name}/Output/{gpd_fc_file_name}"
# Check if the file exist in the bucket. If "Yes", delete
try:
    delete_blob(_bucket_name=bucket_name, _blob_name=destination_blob_name)
except:
    pass
# and upload the file
upload_blob(bucket_name, source_file_name, destination_blob_name)

In [ ]:
# import folium
# m=folium.Map(location=[30.266666, -97.733330])
# display_featureclass = folium.GeoJson(gdf_network.to_json()).add_to(m)
# folium.LayerControl().add_to(m)
# m

In [46]:
# m